In [ ]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

<link rel="stylesheet" href="../../../common/dhds.css">
<div class="Table">
    <div class="Row">
        <div class="Cell grey left"> <img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M6/CLASE_47_Text_Mining_2/Presentacion/img/M6_CLASE_47_portada.jpg" align="center" width="90%"/></div>        
        <div class="Cell right">
            <div class="div-logo"><img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/common/logo_DH.png" align="center" width=70% /></div>
            <div class="div-curso">DATA SCIENCE</div>
            <div class="div-modulo">MÓDULO 6</div>
            <div class="div-contenido">Text Minig <br/> Clasificación de texto</div>
        </div>
    </div>
</div>

## Agenda

---

- Análisis de sentimientos como problema de clasificación

- Clasificación

- N-gramas


<div class="div-dhds-fondo-1"> Introducción
<img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M6/CLASE_47_Text_Mining_2/Presentacion/img/M6_CLASE_47_separador.png" align="center" />

</div>


## Introducción

---

<table><tr><td style="font-size:14px;width:55%;line-height:2;">El análisis de sentimientos es una técnica a través de la cual podemos analizar un fragmento de texto para determinar el sentimiento detrás de él. 
<br/>
Combina el aprendizaje automático y el procesamiento del lenguaje natural (NLP) para lograrlo. 
<br/>
Podemos pensar este problema como un problema de clasificación, donde las features son generadas con el técnicas de preprocesamiento de texto que vimos la clase pasada.</td><td><img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M6/CLASE_47_Text_Mining_2/Presentacion/img/M6_CLASE_47_intro.jpg" align="center"/></td></tr></table>

<div class="div-dhds-fondo-1"> Clasificación
<img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M6/CLASE_47_Text_Mining_2/Presentacion/img/M6_CLASE_47_separador.png" align="center" />

</div>


## Clasificación

---

En esta clase analizaremos un dataset de reviews de imdb (http://ai.stanford.edu/~amaas/data/sentiment/) usando regresión logística para predecir a partir del texto del comentario si la calificación otorgada por el usuario es positiva o negativa.


### Datos

---

Los datos están separados en dos archivos, uno de train y uno de test, cada uno con la misma cantidad de registros.

Las clases están balanceadas tanto en el dataset de train como en el de test, es decir, hay igual cantidad de reviews positivas y negativas.


In [ ]:
train_file_path = '../Data/imdb_train.zip'
data_train = pd.read_csv(train_file_path, sep="\t")
data_train

In [ ]:
data_train.label.value_counts()

In [ ]:
test_file_path = '../Data/imdb_test.zip'
data_test = pd.read_csv(test_file_path, sep="\t")
data_test

In [ ]:
data_test.label.value_counts()

### Imports

In [ ]:
from nltk import RegexpTokenizer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

### Preparación

---

Limpiamos los datos de train y de test:

* usamos un tokennizer que elimine los signos de puntuación y tags html

* hacemos stemming para obtener las raíces de las palabras en minúsculas

* eliminamos stopwords

In [ ]:
def clean_review(review_text, tokenizer, stemmer, stopwords):    
    
    #tokens (eliminamos todos los signos de puntuación)
    words = tokenizer.tokenize(review_text)
    #print(words)
    
    # stemming: raiz y minúsculas:
    stem_words = [stemmer.stem(x) for x in words]
    #print(stem_words)
    
    # eliminamos stopwords (ya pasaron por stem)
    clean_words = [x for x in stem_words if x not in stopwords]
    #print(clean_words)
    
    result = " ".join(clean_words)
    
    return(result)
    

Ejemplo sobre un registro de train 

In [ ]:

review_text = data_train.text[1]

print("antes: ", review_text)

#eliminamos todos los signos de puntuación
tokenizer = RegexpTokenizer(r"\w+")

englishStemmer = SnowballStemmer("english")
stopwords_en = stopwords.words('english');
stopwords_en_stem = [englishStemmer.stem(x) for x in stopwords_en]

review_text_clean = clean_review(review_text, tokenizer, englishStemmer, stopwords_en_stem)


In [ ]:
print("---")
print("después: ", review_text_clean)


Procesamos todos los registros de los datasets de train y test

In [ ]:
clean_train = [clean_review(x, tokenizer, englishStemmer, stopwords_en_stem) for x in data_train.text]
#clean_train[0:5]

In [ ]:
clean_test = [clean_review(x, tokenizer, englishStemmer, stopwords_en_stem) for x in data_test.text]
#clean_test[0:5]


Usando `CountVectorizer` vamos a transformar los datos de train y test

In [ ]:
count_vectorizer = CountVectorizer()
count_vectorizer.fit(clean_train)
X_train_sparse = count_vectorizer.transform(clean_train)
X_test_sparse = count_vectorizer.transform(clean_test)

In [ ]:
X_train = pd.DataFrame(X_train_sparse.todense(), 
             columns = count_vectorizer.get_feature_names()) 
y_train = data_train.label

In [ ]:
X_test = pd.DataFrame(X_test_sparse.todense(), 
             columns = count_vectorizer.get_feature_names()) 
y_test = data_test.label

### Modelo

---

Vamos a usar ahora una regresión logística para predecir el valor de `label`

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split

Veamos qué valor de `C` (regularización) resulta en un modelo de mejor performance

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

Valores más pequeños de `C` implican regularización más fuerte.

In [ ]:
X_train_train, X_train_val, y_train_train, y_train_val = \
    train_test_split(X_train, y_train, train_size = 0.75, shuffle = True, random_state = 147)

for c in [0.005, 0.008, 0.01, 0.05, 0.25, 0.5, 1]:    
    lr = LogisticRegression(C=c, solver="newton-cg", penalty="l2")    
    lr.fit(X_train_train, y_train_train)
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_train_val, lr.predict(X_train_val))))
    

Entrenamos el modelo final

In [ ]:
final_model = LogisticRegression(C = 0.05, solver="newton-cg", penalty="l2")
final_model.fit(X_train, y_train)
print ("Final Accuracy: %s" 
        % accuracy_score(y_test, final_model.predict(X_test)))
print ("Final Confusion Matrix: \n %s" 
        % confusion_matrix(y_test, final_model.predict(X_test)))

Observemos cuáles son las 3 palabras más discriminantes tanto positiva como negativamente. 

Para ver esto, veremos cuáles son las palabras (features) asociadas a los coeficientes máximos y mínimos de la regresión logística.

Construyamos un dataframe de dos columnas:

* word: cada registro tiene un nombre que corresponde a una feature del modelo (las palabras en el corpus)
* coef: cada registro tiene un valor que es el coeficiente de la regresión logística para su feature

Y ordenemos el dataframe según los valores de coef de forma descendente.


In [ ]:
feature_to_coef = pd.DataFrame(columns = ['word', 'coef'])
feature_to_coef.word = count_vectorizer.get_feature_names()
feature_to_coef.coef = final_model.coef_[0]
feature_to_coef_sort_desc = feature_to_coef.sort_values(by = 'coef', ascending = False)
positive_words = feature_to_coef_sort_desc.word[0:3]
positive_words

In [ ]:
feature_to_coef_sort_asc = feature_to_coef.sort_values(by = 'coef', ascending = True)
negative_words = feature_to_coef_sort_asc.word[0:3]
negative_words

Grafiquemos estos resultados para ver cómo se distribuyen los valores

In [ ]:
columns =  np.concatenate([positive_words.values, negative_words.values])
columns

In [ ]:
data_plot = X_train.loc[:, columns]
data_plot.reset_index(inplace = True)
data_plot["label"] = y_train

In [ ]:
data_plot_long = pd.melt(data_plot, 
                         id_vars = ["index", "label"], 
                         #value_vars = data_plot.columns[1:len(data_plot.columns)-1], 
                         var_name = "word", value_name = "value")
data_plot_long["word_label"] = data_plot_long.word + ["_" + str(x) for x in data_plot_long.label]  
data_plot_long

In [ ]:
data_plot_long.sort_values(by = "index").head()

In [ ]:
plt.figure(figsize=(15,10))
sns.violinplot(x = data_plot_long.word, y = data_plot_long.value, hue = data_plot_long.label);

### Singular Value Decomposition

---

Entrenemos una regresión logística que use como features el resultado de SVD.

In [ ]:
feature_to_coef.shape

El modelo anterior fue entrenado con más de 50 mil features!

Veamos cómo cambia la performance seleccionando 200 features.

In [ ]:

from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components = 200);

X_train_svd = svd.fit_transform(X_train)


In [ ]:
X_test_svd = svd.transform(X_test)

In [ ]:
X_train_svd_train, X_train_svd_val, y_train_train, y_train_val = \
    train_test_split(X_train_svd, y_train, train_size = 0.75, shuffle = True, random_state = 147)

for c in [0.005, 0.008, 0.009, 0.01, 0.05, 0.07]:    
    lr = LogisticRegression(C=c, solver="newton-cg", penalty="l2")    
    lr.fit(X_train_svd_train, y_train_train)
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_train_val, lr.predict(X_train_svd_val))))
    

In [ ]:
final_model_svd = LogisticRegression(C = 0.01, solver="newton-cg", penalty="l2")
final_model_svd.fit(X_train_svd, y_train)
print ("Final Accuracy: %s" 
       % accuracy_score(y_test, final_model_svd.predict(X_test_svd)))
print ("Final Confusion Matrix: \n %s" 
        % confusion_matrix(y_test, final_model_svd.predict(X_test_svd)))

## N-gramas

---

Hasta ahora usamos sólo features compuestas por una sola palabra en nuestro modelo, que llamamos **1-grama** o **unigrama**. 

Potencialmente, podemos agregar más poder predictivo a nuestro modelo agregando también **secuencias de dos o tres palabras (bigramas o trigramas)**. 

Por ejemplo, si una reseña tuviera la secuencia de tres palabras "no me gustó la película", solo consideraríamos estas palabras individualmente con un modelo de unigramas y probablemente no captaríamos que se trata de un sentimiento negativo porque la palabra "gustó" en sí misma va a estar altamente correlacionado con una revisión positiva.

Scikit-learn hace que sea muy fácil construir estas features. 

Simplemente usamos el argumento `ngram_range` con cualquiera de las clases "Vectorizador".

In [ ]:
count_vectorizer_bigram = CountVectorizer(ngram_range = (1, 2))
count_vectorizer_bigram.fit(clean_train)
X_train_bigram_sparse = count_vectorizer_bigram.transform(clean_train)
X_test_bigram_sparse = count_vectorizer_bigram.transform(clean_test)

#X_train_bigram = pd.DataFrame(X_train_bigram_sparse.todense(), 
#             columns = count_vectorizer_bigram.get_feature_names()) 
             
#X_test_bigram = pd.DataFrame(X_test_bigram_sparse.todense(), 
#             columns = count_vectorizer_bigram.get_feature_names()) 

#usamos las matrices esparsas porque rompe si trato de convertrlas en densas para esta cantidad de features
X_train_bigram_train, X_train_bigram_val, y_train_train, y_train_val = \
    train_test_split(X_train_bigram_sparse, y_train, train_size = 0.75, shuffle = True, random_state = 147)

for c in [0.01, 0.05, 0.1, 0.25, 0.5, 0.6, 0.7, 0.8, 0.9, 1]:    
    lr = LogisticRegression(C=c, solver="newton-cg", penalty="l2")    
    lr.fit(X_train_bigram_train, y_train_train)
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_train_val, lr.predict(X_train_bigram_val))))


In [ ]:
final_model_bigram = LogisticRegression(C = 0.25, solver="newton-cg", penalty="l2")
final_model_bigram.fit(X_train_bigram_sparse, y_train)
print ("Final Accuracy: %s" 
       % accuracy_score(y_test, final_model_bigram.predict(X_test_bigram_sparse)))    
print ("Final Confusion Matrix: \n %s" 
        % confusion_matrix(y_test, final_model_bigram.predict(X_test_bigram_sparse)))

<div class="div-dhds-fondo-1"> Conclusiones
<img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M6/CLASE_47_Text_Mining_2/Presentacion/img/M6_CLASE_47_separador.png" align="center" />

</div>

* Construimos un modelo de clasificación de textos empleando los algoritmos que ya conocíamos.

* Construimos el conjunto de features del modelo transformando los inputs (textos) mediante las técnicas de preprocesamiento que vimos en la clase pasada.

* Evaluamos la performance del modelo del mismo modo que lo hacemos con cualquier modelo de clasificación.


<div class="div-dhds-fondo-1"> Hands-on
<img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M6/CLASE_47_Text_Mining_2/Presentacion/img/M6_CLASE_47_separador.png" align="center" />

</div>


## Ejercicio

---

TF-IDF es mejor que CountVectorizer porque no sólo se centra en la frecuencia de las palabras presentes en el corpus, sino que también tiene en cuenta su importancia. 

Volvamos a entrenar una regresión logísticas usando como features la representación tf-idf de unigramas, bigramas y trigramas.

Veamos cuáles son los n-gramas más discriminantes.


## Solución

---

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
count_vectorizer = CountVectorizer(ngram_range = (1, 3))
count_vectorizer.fit(clean_train)
X_train_sparse = count_vectorizer.transform(clean_train)
X_test_sparse = count_vectorizer.transform(clean_test)

In [ ]:
transformer = TfidfTransformer()

X_train_ngram_encoding = transformer.fit_transform(X_train_sparse);

X_test_ngram_encoding = transformer.transform(X_test_sparse);


In [ ]:
X_train_ngram_train, X_train_ngram_val, y_train_train, y_train_val = \
    train_test_split(X_train_ngram_encoding, y_train, train_size = 0.75, shuffle = True, random_state = 147)

for c in [0.05, 0.25, 0.5, 0.7, 0.8, 0.9, 1]:    
    lr = LogisticRegression(C=c, solver="newton-cg", penalty="l2")    
    lr.fit(X_train_ngram_train, y_train_train)
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_train_val, lr.predict(X_train_ngram_val))))


In [ ]:
final_model_tfidf = LogisticRegression(C = 1, solver="newton-cg", penalty="l2")
final_model_tfidf.fit(X_train_ngram_encoding, y_train)
print ("Final Accuracy: %s" 
       % accuracy_score(y_test, final_model_tfidf.predict(X_test_ngram_encoding)))
print ("Final Confusion Matrix: \n %s" 
        % confusion_matrix(y_test, final_model_tfidf.predict(X_test_ngram_encoding)))

In [ ]:
feature_to_coef = pd.DataFrame(columns = ['ngram', 'coef'])
feature_to_coef.ngram = count_vectorizer.get_feature_names()
feature_to_coef.coef = final_model_tfidf.coef_[0]
feature_to_coef_sort_desc = feature_to_coef.sort_values(by = 'coef', ascending = False)
feature_to_coef_sort_desc.ngram[0:5]

In [ ]:
feature_to_coef_sort_asc = feature_to_coef.sort_values(by = 'coef', ascending = True)
feature_to_coef_sort_asc.ngram[0:5]


Vemos que los n-grams más discriminantes son unigramas

<div class="div-dhds-fondo-1"> Referencias y material adicional
<img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M6/CLASE_47_Text_Mining_2/Presentacion/img/M6_CLASE_47_separador.png" align="center" />

</div>

<a href="https://towardsdatascience.com/how-to-build-a-twitter-sentiment-analysis-system-12b28dcbae56" target="_blank">How to Build a Twitter Sentiment Analysis System</a>

<a href="https://towardsdatascience.com/sentiment-analysis-with-python-part-1-5ce197074184" target="_blank">Sentiment Analysis with Python (Part 1)</a>

<a href="https://towardsdatascience.com/sentiment-analysis-with-python-part-2-4f71e7bde59a" target="_blank">Sentiment Analysis with Python (Part 2)</a>

<a href="https://towardsdatascience.com/imdb-reviews-or-8143fe57c825" target="_blank">Performing Sentiment Analysis on Movie Reviews</a>

<a href="https://www.kaggle.com/lakshmi25npathi/sentiment-analysis-of-imdb-movie-reviews" target="_blank">Sentiment Analysis of IMDB Movie Reviews</a>  
        